In [8]:
import pandas as pd
import glob
from sklearn.model_selection import train_test_split
from datasets import Dataset

In [105]:
images = glob.glob('/mnt/nas05/data01/vincenzo/ecallisto/data/*/*/*/*.png')

In [106]:
df = pd.DataFrame()

In [108]:
df['file_name'] = images
df['label'] = df['file_name'].str.split('/', expand=True)[9].astype(str)
df['antenna'] = df['file_name'].str.split('/', expand=True)[8].astype(str)
df['datetime'] = df['file_name'].str.split('/', expand=True)[10].str.replace('.png', '')
df['file_name'] = df['file_name'].str.replace('/mnt/nas05/data01/vincenzo/ecallisto/data/', '').astype(str)
df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d_%H-%M-%S')#.astype('int64') // 10**9

In [119]:
train, val, test = df[df['file_name'].str.contains('train')],  df[df['file_name'].str.contains('valid')],  df[df['file_name'].str.contains('test')]
train['file_name'] = train['file_name'].str.replace('train/', '').astype(str)
val['file_name'] = val['file_name'].str.replace('valid/', '').astype(str)
test['file_name'] = test['file_name'].str.replace('test/', '').astype(str)

/tmp/ipykernel_1707095/2757098019.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['file_name'] = train['file_name'].str.replace('train/', '').astype(str)
/tmp/ipykernel_1707095/2757098019.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val['file_name'] = val['file_name'].str.replace('valid/', '').astype(str)
/tmp/ipykernel_1707095/2757098019.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


In [121]:
train.to_csv('/mnt/nas05/data01/vincenzo/ecallisto/data/train/metadata.csv', index=False)
test.to_csv('/mnt/nas05/data01/vincenzo/ecallisto/data/test/metadata.csv', index=False)
val.to_csv('/mnt/nas05/data01/vincenzo/ecallisto/data/valid/metadata.csv', index=False)

In [141]:
test

DatasetDict({
    train: Dataset({
        features: ['image', 'label', 'antenna', 'datetime'],
        num_rows: 9010
    })
})

In [ ]:
# EDA
import matplotlib.pyplot as plt
# Number of images per antenna
images_per_antenna = test['antenna'].value_counts()

# Number of burst types per antenna
burst_types_per_antenna = test.groupby('antenna')['label'].value_counts().unstack(fill_value=0)

# Date range per antenna
date_range_per_antenna = test.groupby('antenna')['datetime'].agg([min, max])

# Plotting
fig, ax = plt.subplots(2, 1, figsize=(10, 15))

# Images per antenna
images_per_antenna.plot(kind='bar', ax=ax[0], color='skyblue')
ax[0].set_title('Number of Images per Antenna')
ax[0].set_xlabel('Antenna')
ax[0].set_ylabel('Number of Images')

# Burst types per antenna
burst_types_per_antenna.plot(kind='bar', stacked=True, ax=ax[1])
ax[1].set_title('Burst Types per Antenna')
ax[1].set_xlabel('Antenna')
ax[1].set_ylabel('Count')

plt.tight_layout()
plt.show()

In [123]:
from datasets import load_dataset
test = load_dataset('imagefolder', data_dir='/mnt/nas05/data01/vincenzo/ecallisto/data/test')
valid = load_dataset('imagefolder', data_dir='/mnt/nas05/data01/vincenzo/ecallisto/data/valid')
train = load_dataset('imagefolder', data_dir='/mnt/nas05/data01/vincenzo/ecallisto/data/train')

Generating train split: 9010 examples [00:04, 1824.83 examples/s]
Generating train split: 9010 examples [00:04, 1829.75 examples/s]
Computing checksums: 100%|██████████| 72124/72124 [01:28<00:00, 818.91it/s]
Generating train split: 72123 examples [00:42, 1696.27 examples/s]


In [134]:
from datasets import DatasetDict
dd = DatasetDict({"train": train['train'],"validation": valid['train'], "test":test['train']})

In [139]:
dd.push_to_hub('i4ds/radio-sunburst-ecallisto')

Uploading the dataset shards: 100%|██████████| 1/1 [00:43<00:00, 43.25s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/i4ds/radio-sunburst-ecallisto/commit/eb681e93a33694c0063413948282162926537383', commit_message='Upload dataset', commit_description='', oid='eb681e93a33694c0063413948282162926537383', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# Display the image
plt.imshow(dd['train']['image'][300], cmap='gray')  # 'gray' colormap for mode=L (grayscale)
plt.title(f"Antenna: {dd['train']['antenna'][300]} | Datetime: {dd['train']['datetime'][300]} | Label: {dd['train']['label'][300]}")
plt.axis('off')  # Turn off the axis
plt.show()